In [1]:
pip install scikit-learn rdkit-pypi chython pandas numpy torch pytorch-lightning mordred

Note: you may need to restart the kernel to use updated packages.


In [5]:
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from typing import Any, List, Callable, Optional

import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy
from rdkit import Chem, RDLogger
from rdkit.Chem import Descriptors, rdMolDescriptors, Lipinski, MolFromSmiles, rdFingerprintGenerator as fp
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem.Draw import IPythonConsole
RDLogger.DisableLog('rdApp.*')  # чтобы не выскакивали сообщения об ошибке

from mordred import Calculator, descriptors
IPythonConsole.ipython_useSVG = True  # set this to False if you want PNGs instead of SVGs


from chython import smiles
from collections import defaultdict
from collections.abc import Iterable

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import pytorch_lightning as pl

from torch.utils.data import Dataset, DataLoader, TensorDataset

import random
import warnings
import os
import pickle

warnings.filterwarnings("ignore") 
nproc = os.cpu_count() - 1 
nproc

3

## Данные

In [ ]:
train_data = pd.read_csv('/kaggle/input/sibur119cava80/sibur_element_119_final_train_data80.csv') #change to yours
test_data = pd.read_csv('/kaggle/input/sibur119cava80/sibur_element_119_final_test_data80.csv') #change to yours

In [7]:
def is_valid_smiles(s):
  return isinstance(s, str) and s.strip() != "" and Chem.MolFromSmiles(s) is not None

def filter_invalid_smiles(df):
  return df[df['SMILES'].apply(is_valid_smiles)].reset_index(drop=True)

train_data = filter_invalid_smiles(train_data)
test_data = filter_invalid_smiles(test_data)

In [22]:
# Функция для мутации SMILES с проверкой на валидность молекулы
def mutate_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None or not mol.HasSubstructMatch(mol):  # Проверка на валидность молекулы
        return None  # Возвращаем None, если молекула некорректна
    
    # Пример мутации: заменяем метильную группу на этильную (можно придумать другие)
    mutated_smiles = Chem.MolToSmiles(mol)
    mutated_smiles = mutated_smiles.replace('C', 'CC', 1)  # Пример замены, можно настроить
    return mutated_smiles

def mutate_smiles_3(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None or not mol.HasSubstructMatch(mol):  # Проверка на валидность молекулы
        return None  # Возвращаем None, если молекула некорректна
    
    # Пример мутации: заменяем метильную группу на этильную (можно придумать другие)
    mutated_smiles = Chem.MolToSmiles(mol)
    mutated_smiles = mutated_smiles.replace('OC', 'OCC', 1)  # Пример замены, можно настроить
    return mutated_smiles

# Функция для добавления шума в LogP
def add_noise_logp(logp_value, noise_level=0.1):
    return logp_value + random.uniform(-noise_level, noise_level)

# Функция для аугментации данных в DataFrame с проверкой на валидность молекулы
def augment_data(df, noise_level=0.1, num_augments=1):
    augmented_data = []

    for idx, row in df.iterrows():
        smiles = row['SMILES']
        logp = row['LogP']
        
        # Добавляем оригинальную строку в результат
        augmented_data.append((smiles, logp))

        # Создаём аугментированные строки
        mutated_smiles = mutate_smiles(smiles)
        mutated_smiles_3 = mutate_smiles_3(smiles)
        if (mutated_smiles != None) and (MolFromSmiles(str(mutated_smiles)) != None):
            noisy_logp = add_noise_logp(logp, noise_level)
            augmented_data.append((mutated_smiles, noisy_logp))
        if (mutated_smiles_3 != None) and (MolFromSmiles(str(mutated_smiles_3)) != None):
            noisy_logp = add_noise_logp(logp, noise_level)
            augmented_data.append((mutated_smiles_3, noisy_logp))
    
    # Создаём новый DataFrame с аугментированными данными
    augmented_df = pd.DataFrame(augmented_data, columns=['SMILES', 'LogP'])

    # Убираем дубликаты, оставляя только уникальные SMILES с одним значением LogP
    augmented_df = augmented_df.drop_duplicates(subset=['SMILES'])

    return augmented_df

# Аугментация данных
augmented_df = augment_data(train_data, noise_level=0.15, num_augments=2)
len(augmented_df)
train_data=augmented_df

In [23]:
n = int(len(train_data)*0.2) # размер валидационной выборки
random_indices = np.random.choice(train_data.index, size=n, replace=False) # случайне индексы
val_data = train_data.loc[random_indices]
train_data_remain = train_data.drop(random_indices)

print(f"Исходный DataFrame: {len(train_data)} строк")
print(f"Удалено строк: {len(val_data)}")
print(f"Осталось строк: {len(train_data_remain)}")
train_data = train_data_remain
print(val_data.head())

Исходный DataFrame: 30986 строк
Удалено строк: 6197
Осталось строк: 24789
                                       SMILES      LogP
36119           C=1(C(=CC=CC=1)[N+](=O)[O-])C  3.605000
25318             CCCCN(CCC)C1COc2ccc(O)cc2C1  4.996713
3395   NC(=O)c1cccc(OCC(=O)CNC(=O)c2ccccc2)c1  2.649671
22282               C1=C2C(=O)N(O)C=NC2=CC=C1  0.446000
32776                    Clc1ccc(Cn2cncn2)cc1  3.327556


In [7]:
# вычленение текста смайлс в массив
test_smiles_list = test_data['SMILES'].tolist()
train_smiles_list = train_data['SMILES'].tolist()
val_smiles_list = val_data['SMILES'].tolist()

# перевод текста смайлс в смайлс тип
train_mols = [smiles(m) for m in train_smiles_list]
test_mols = [smiles(m) for m in test_smiles_list]
val_mols = [smiles(m) for m in val_smiles_list]

In [8]:
# стандартизация молекул

def standardize(mol_list):
    for m in mol_list:
        try:
            m.clean_stereo()
            m.canonicalize()
        except:
            print(m)
standardize(train_mols)
standardize(test_mols)
standardize(val_mols)

In [9]:
# перевод смайлс в молекулу

train_rdkit_mols = [MolFromSmiles(str(m)) for m in train_mols]
test_rdkit_mols = [MolFromSmiles(str(m)) for m in test_mols]
val_rdkit_mols = [MolFromSmiles(str(m)) for m in val_mols]

In [10]:
# число проблемных молекул итоговое

sum(1 for mol in train_rdkit_mols if mol is None)

0